# 1- Data Preparation:

In [1]:
import pandas as pd
import random

# Charger les données à partir des fichiers CSV
train_df = pd.read_csv('train.csv')

# Créer une liste de tous les utilisateurs uniques
unique_users = train_df['user'].unique()

# Initialiser la liste pour stocker les paires de tweets
tweet_pairs = []

# Déterminer le nombre maximum de paires à générer pour chaque utilisateur
max_pairs_per_user = len(train_df) // len(unique_users)

# Boucler à travers chaque utilisateur
for user in unique_users:
    # Extraire tous les tweets de l'utilisateur actuel
    user_tweets = train_df[train_df['user'] == user]['text'].tolist()
    
    # Générer des paires de tweets à partir du même utilisateur
    same_user_pairs = random.sample(list(zip(user_tweets, user_tweets)), max_pairs_per_user)
    
    # Extraire les tweets des autres utilisateurs
    other_users_tweets = train_df[train_df['user'] != user]['text'].tolist()
    
    # Sélectionner aléatoirement un nombre équivalent de tweets des autres utilisateurs
    random_other_tweets = random.sample(other_users_tweets, max_pairs_per_user)
    
    # Créer des paires de tweets avec des utilisateurs différents
    different_user_pairs = list(zip(user_tweets, random_other_tweets))
    
    # Ajouter les paires de tweets à la liste avec une étiquette de similarité 1 pour les mêmes utilisateurs
    tweet_pairs.extend([[pair[0], pair[1], 1] for pair in same_user_pairs])
    
    # Ajouter les paires de tweets à la liste avec une étiquette de similarité 0 pour les différents utilisateurs
    tweet_pairs.extend([[pair[0], pair[1], 0] for pair in different_user_pairs])

# Mélanger les paires de tweets pour assurer un ordre aléatoire
random.shuffle(tweet_pairs)

# Convertir en DataFrame
tweet_pairs_df = pd.DataFrame(tweet_pairs, columns=['tweet1', 'tweet2', 'similarity_label'])

# Afficher un aperçu du DataFrame
print(tweet_pairs_df.head())

# Enregistrer le DataFrame dans un fichier CSV
tweet_pairs_df.to_csv('balanced_tweet_pairs.csv', index=False)

                                              tweet1  \
0             #CouchSelfiepic.twitter.com/AZDIB88OwR   
1  New GOP platform now includes language that su...   
2  Glad to contribute to the Tesla museum and wil...   
3  #PrayForSyria watch this and think like a chil...   
4  NY! #25FOR1 Auction tonight at @No8NY with all...   

                                              tweet2  similarity_label  
0             #CouchSelfiepic.twitter.com/AZDIB88OwR                 1  
1  President Obama on Romney: "We know what chang...                 0  
2  Glad to contribute to the Tesla museum and wil...                 1  
3  tweeted July 30 2011 - "what makes you so beau...                 0  
4  Keep sending them to weare50million@gmail.com ...                 0  


# 2- Data Preprocessing:

In [3]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

# Télécharger les ressources nécessaires pour NLTK
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Charger le DataFrame
df = pd.read_csv('balanced_tweet_pairs.csv')

# Initialiser le lemmatiseur
lemmatizer = WordNetLemmatizer()

# Fonction de nettoyage du texte avec lemmatisation
def clean_text(text):
    # Convertir en minuscules
    text = text.lower()
    # Supprimer la ponctuation, les symboles et les autres caractères spéciaux sauf les hashtags et les mentions
    text = ''.join([char if char in ['#', '@'] or char not in string.punctuation else ' ' for char in text])
    # Tokenization
    tokens = word_tokenize(text)
    # Supprimer les mots vides (stop words)
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatisation
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    # Rejoindre les tokens pour former une chaîne de texte
    cleaned_text = ' '.join(lemmatized_tokens)
    return cleaned_text

# Appliquer la fonction de nettoyage du texte à chaque tweet dans le DataFrame
df['tweet1'] = df['tweet1'].apply(clean_text)
df['tweet2'] = df['tweet2'].apply(clean_text)

# Afficher un aperçu du DataFrame après le prétraitement
print(df.head())

# Enregistrer le DataFrame prétraité dans un nouveau fichier CSV
df.to_csv('preprocessed_balanced_tweet_pairs.csv', index=False)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\khadidja\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\khadidja\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\khadidja\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


                                              tweet1  \
0            # couchselfiepic twitter com azdib88owr   
1  new gop platform includes language support bor...   
2  glad contribute tesla museum future great man ...   
3  # prayforsyria watch think like child love ans...   
4  ny # 25for1 auction tonight @ no8ny proceeds g...   

                                              tweet2  similarity_label  
0            # couchselfiepic twitter com azdib88owr                 1  
1  president obama romney know change look like s...                 0  
2  glad contribute tesla museum future great man ...                 1  
3  tweeted july 30 2011 make beautiful dont know ...                 0  
4  keep sending weare50million @ gmail com soon s...                 0  


# 3- Model Architecture and Evaluation:

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from keras.models import Model
from keras.layers import Input, Subtract, Dense, Lambda
from keras import backend as K
from sklearn.metrics import precision_score, recall_score, f1_score


# Chargement des données
df = pd.read_csv('preprocessed_balanced_tweet_pairs.csv')


# Fonction pour prétraiter les tweets
def preprocess_tweet(tweet):
    return tweet.split()

# Remplacer les valeurs NaN par une chaîne vide
df['tweet1'] = df['tweet1'].fillna('')
df['tweet2'] = df['tweet2'].fillna('')

# Prétraitement des tweets et création des listes de listes de mots
df['tweet1'] = df['tweet1'].apply(preprocess_tweet)
df['tweet2'] = df['tweet2'].apply(preprocess_tweet)

# Entraînement du modèle Word2Vec 
embedding_dim = 100  # Taille de l'embedding
word2vec_model = Word2Vec(sentences=df['tweet1'].tolist() + df['tweet2'].tolist(),
                          vector_size=embedding_dim, window=5, min_count=1, workers=4)

# Fonction pour obtenir la représentation vectorielle d'un tweet à partir du modèle Word2Vec
def get_tweet_embedding(tweet, model):
    embedding = []
    for word in tweet:
        if word in model.wv:
            embedding.append(model.wv[word])
    if embedding:
        return np.mean(embedding, axis=0)
    else:
        return np.zeros(embedding_dim)

# Obtenir les représentations vectorielles des tweets
df['tweet1_embedding'] = df['tweet1'].apply(lambda x: get_tweet_embedding(x, word2vec_model))
df['tweet2_embedding'] = df['tweet2'].apply(lambda x: get_tweet_embedding(x, word2vec_model))

# Fonction pour calculer la distance de Manhattan entre deux vecteurs
def manhattan_distance(vec1, vec2):
    return K.sum(K.abs(vec1 - vec2), axis=-1, keepdims=True)

# Inputs
input1 = Input(shape=(embedding_dim,))
input2 = Input(shape=(embedding_dim,))

# Manhattan Distance
distance = Lambda(lambda tensors: manhattan_distance(tensors[0], tensors[1]))([input1, input2])

# Dense Layer
output = Dense(1, activation='sigmoid')(distance)

# Model
model = Model(inputs=[input1, input2], outputs=output)

# Compile
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Entraînement du modèle
model.fit([np.array(df['tweet1_embedding'].tolist()), np.array(df['tweet2_embedding'].tolist())],
          df['similarity_label'], epochs=10, batch_size=32, validation_split=0.1)




# Charger les données de test
test_df = pd.read_csv('preprocessed_test_tweet_pairs.csv')

# Prétraiter les tweets de l'ensemble de test 
test_df['tweet1'] = test_df['tweet1'].fillna('').apply(preprocess_tweet)
test_df['tweet2'] = test_df['tweet2'].fillna('').apply(preprocess_tweet)

# Obtenir les représentations vectorielles des tweets pour l'ensemble de test 
test_df['tweet1_embedding'] = test_df['tweet1'].apply(lambda x: get_tweet_embedding(x, word2vec_model))
test_df['tweet2_embedding'] = test_df['tweet2'].apply(lambda x: get_tweet_embedding(x, word2vec_model))

# Évaluation du modèle
loss, accuracy = model.evaluate([np.array(test_df['tweet1_embedding'].tolist()), np.array(test_df['tweet2_embedding'].tolist())],
                                test_df['similarity_label'])
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Prédictions sur l'ensemble de test
predictions = model.predict([np.array(test_df['tweet1_embedding'].tolist()), np.array(test_df['tweet2_embedding'].tolist())])
predictions = (predictions > 0.5).astype(int)


# Labels réels
true_labels = test_df['similarity_label'].values

# Calcul de la précision
precision = precision_score(true_labels, predictions)
print("Precision:", precision)

# Calcul du rappel
recall = recall_score(true_labels, predictions)
print("Recall:", recall)

# Calcul du score F1
f1 = f1_score(true_labels, predictions)
print("F1 Score:", f1)

Epoch 1/10
1170/1170 [==============================] - 5s 3ms/step - loss: 13.5993 - accuracy: 0.0211 - val_loss: 0.7499 - val_accuracy: 0.4945
Epoch 2/10
1170/1170 [==============================] - 4s 3ms/step - loss: 0.4390 - accuracy: 0.6752 - val_loss: 0.2828 - val_accuracy: 1.0000
Epoch 3/10
1170/1170 [==============================] - 4s 3ms/step - loss: 0.2021 - accuracy: 0.9999 - val_loss: 0.1426 - val_accuracy: 1.0000
Epoch 4/10
1170/1170 [==============================] - 4s 3ms/step - loss: 0.1061 - accuracy: 0.9999 - val_loss: 0.0779 - val_accuracy: 1.0000
Epoch 5/10
1170/1170 [==============================] - 4s 3ms/step - loss: 0.0597 - accuracy: 0.9998 - val_loss: 0.0449 - val_accuracy: 1.0000
Epoch 6/10
1170/1170 [==============================] - 5s 4ms/step - loss: 0.0350 - accuracy: 0.9998 - val_loss: 0.0266 - val_accuracy: 1.0000
Epoch 7/10
1170/1170 [==============================] - 4s 4ms/step - loss: 0.0211 - accuracy: 0.9998 - val_loss: 0.0161 - val_accuracy